In [ ]:
#import library
import cv2
from torchvision import transforms, models, datasets
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt
#build cascade obj
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
#define vgg16 model
def get_model():
    model = models.vgg19(pretrained = True)
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.Sequential(
        nn.Conv2d(512,512, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten()
    )
    class ageGenderClassifier(nn.Module):
        def __init__(self):
            super(ageGenderClassifier, self).__init__()
            self.intermediate = nn.Sequential(
                nn.Linear(2048,512),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(512,128),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(128,64),
                nn.ReLU(),
            )
            self.age_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
            self.gender_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
        def forward(self, x):
            x = self.intermediate(x)
            age = self.age_classifier(x)
            gender = self.gender_classifier(x)
            return gender, age

    model.classifier = ageGenderClassifier()

    return model.to(device)


In [ ]:
def model_prediction(pre_img, model):

    face = pre_img.to(device).float()
    gender, age = model(face)

    pred_gender = gender.to('cpu').detach().numpy()
    pred_age = age.to('cpu').detach().numpy()

    return pred_gender, pred_age

In [ ]:
import torch

def detection(gray):

    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    # model = torch.hub.load('ultralytics/yolov5', model_name, source=model_url)
    result = model(gray)

    face = result.pandas().xyxy[0]
    return face

In [ ]:
# def preprocess_image(face, frame):
#     # Iterate over the rows of the DataFrame and extract numerical values
#     preprocessed_images = []
#     for index, row in face.iterrows():
#         frame = np.array(frame)
#         x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
#         crop_frame = frame[y1:y2, x1:x2]

#         crop_frame = cv2.resize(crop_frame, (224, 224))
#         crop_frame = torch.tensor(crop_frame).permute(2,0,1)
#         crop_frame = normalize(crop_frame/255.)

#         preprocessed_images.append(crop_frame[None])

#     # Return a list of preprocessed images or None if no faces detected
#         return preprocessed_images if preprocessed_images else None

#preprocess function#preprocess function
def preprocess_image(face, frame):
    # Iterate over the rows of the DataFrame and extract numerical values
    for index, row in face.iterrows():
        frame = np.array(frame)
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        crop_frame = frame[y1:y2, x1:x2]

        crop_frame = cv2.resize(crop_frame, (224, 224))
        crop_frame = torch.tensor(crop_frame).permute(2,0,1)
        crop_frame = normalize(crop_frame/255.)

        return crop_frame[None]
# def preprocess_image(face, frame):
#     for x1, y1, x2, y2 in face:
#         frame = np.array(frame)
#         x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
#         crop_frame = frame[y1:y2, x1:x2]

#         crop_frame = cv2.resize(crop_frame, (224, 224))
#         crop_frame = torch.tensor(crop_frame).permute(2,0,1)
#         crop_frame = normalize(crop_frame/255.)

#         return crop_frame[None]

In [ ]:
cap = cv2.VideoCapture('/content/Faces.mp4')

print(cap.isOpened())
model = get_model()
model.eval()

In [ ]:
#Capture video from webcam
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Initialize video writer.
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)

while cap.isOpened():

    success, frame = cap.read()

    if not success:
        print("Null.Frames")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    detect_face = detection(gray)

    pre_img = preprocess_image(detect_face, frame)
    gender, age = model_prediction(pre_img, model)

    gender = np.where(gender[0][0]<0.5,'Male','Female')
    age = int(age[0][0]*116)

    for _, row in detect_face.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        cv2.rectangle(frame, (x1,y1), (x2, y2), (100, 50, 200), 3)

        cv2.putText(frame, f'Gender: {gender}, Age: {age}', (x1, y2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 255, 255), 1, cv2.LINE_AA)

    #    video_output.write(image)

    video_output.write(frame)

cap.release()
video_output.release()